In [1]:
from flask import Flask, request
from flask_restful import Resource, Api
from flask_cors import CORS
from sqlalchemy import create_engine
from json import dumps
from flask import jsonify, redirect
import optparse
import stardog

app = Flask(__name__)
cors = CORS(app, resources={r"*": {"origins": "*"}}) # CORS
api = Api(app)

conn_details = {
    'endpoint': 'http://webengineering.ins.hs-anhalt.de:32196',
    'username': 'admin',
    'password': 'admin'
}

dbName = 'db1'

In [2]:
# Klasse um Daten aus Stardog zu holen
class getData(Resource):
    def post(self):
        if not request.json:
            return jsonify({'status': 'could not get requested data', 'statuscode': 409})
        else:
            print(request.json)
            with stardog.Connection(dbName, **conn_details) as conn:
                conn.begin()
                results = conn.select('select * { ?a ?p ?o }')
                print(results)
            return jsonify({'status': 'updated highscore with id: ', 'statuscode': 200})

In [ ]:
# Klassen an entsprechende URLs binden
api.add_resource(getData, '/universities')

# Run für Testzwecke, startet auf localhost:5000
if __name__ == "__main__":
    app.run(port=4000)

# Run für Dockercontainer, mit Portangabe
#if __name__ == "__main__":
 #   parser = optparse.OptionParser(usage="python server.py -p ")
  #  parser.add_option('-p', '--port', action='store', dest='port', help='The port to listen on.')
   # (args, _) = parser.parse_args()
    #if args.port == None:
     #   print("Missing required argument: -p/--port")
      #  sys.exit(1)
    #app.run(host='0.0.0.0', port=int(args.port), debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2019 13:37:20] "OPTIONS /universities HTTP/1.1" 200 -


{'uniName': '', 'subject': '', 'extraOptions': [{'count': '', 'type': 'restaurant', 'perimeter': ''}, {'count': '', 'type': 'restaurant', 'perimeter': ''}, {'count': '', 'type': 'restaurant', 'perimeter': ''}]}


127.0.0.1 - - [07/Sep/2019 13:37:21] "POST /universities HTTP/1.1" 200 -


{'head': {'vars': ['a', 'p', 'o']}, 'results': {'bindings': [{'p': {'type': 'uri', 'value': 'http://schema.org/geo'}, 'a': {'type': 'bnode', 'value': 'genid-ababf8521fc645d890e9a9074bbdf295-b0'}, 'o': {'type': 'bnode', 'value': 'genid-ababf8521fc645d890e9a9074bbdf295-b1'}}, {'p': {'type': 'uri', 'value': 'http://schema.org/name'}, 'a': {'type': 'bnode', 'value': 'genid-ababf8521fc645d890e9a9074bbdf295-b0'}, 'o': {'type': 'literal', 'value': 'Freie Universität Berlin'}}, {'p': {'type': 'uri', 'value': 'https://schema.org/hasOfferCatalog'}, 'a': {'type': 'bnode', 'value': 'genid-ababf8521fc645d890e9a9074bbdf295-b0'}, 'o': {'type': 'literal', 'value': 'Allgemeine und Vergleichende Literaturwissenschaft'}}, {'p': {'type': 'uri', 'value': 'https://schema.org/hasOfferCatalog'}, 'a': {'type': 'bnode', 'value': 'genid-ababf8521fc645d890e9a9074bbdf295-b38'}, 'o': {'type': 'literal', 'value': 'Allgemeine und Vergleichende Literaturwissenschaft'}}, {'p': {'type': 'uri', 'value': 'https://schema.o